# LLM Search Agent Notebook
This notebook demonstrates a simple LLM-powered search agent that fetches information from multiple websites and summarizes it.

## Install Required Packages
Make sure you have the required libraries installed. Run this cell if needed.

In [ ]:
!pip install openai beautifulsoup4 requests

## Import Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI

## Define LLM Search Agent Class

In [ ]:
class LLMSearchAgent:
    def __init__(self, model="gpt-4o-mini"):
        self.model = model
        self.client = OpenAI()
    
    def fetch_web_text(self, url: str) -> str:
        try:
            html = requests.get(url, timeout=10).text
            soup = BeautifulSoup(html, "html.parser")
            paragraphs = [p.get_text() for p in soup.find_all("p")]
            return " ".join(paragraphs)
        except Exception as e:
            return f"Error fetching {url}: {e}"

    def search_and_summarize(self, query: str, urls: list, top_k: int = 3) -> str:
        combined_text = ""
        for url in urls[:top_k]:
            text = self.fetch_web_text(url)
            combined_text += f'\n\nSource: {url}\n{text}'
        prompt = f'''
        You are an expert search assistant.
        Summarize the most important information related to the query: "{query}"
        from the following sources. Include main points, insights, and relevant details.
        Sources:
        {combined_text}
        '''
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4
        )
        return response.choices[0].message.content.strip()

## Example Usage

In [ ]:
agent = LLMSearchAgent()
urls_to_search = [
    "https://en.wikipedia.org/wiki/Artificial_intelligence",
    "https://www.britannica.com/technology/artificial-intelligence",
    "https://www.sciencedaily.com/news/computers_math/artificial_intelligence/"
]
query = "Applications of AI in healthcare"
summary = agent.search_and_summarize(query, urls_to_search)
summary